Problem : How to use python notebook to call bigquery?

In [26]:
# from google.cloud import bigquery

# # Create a BigQuery client
# client = bigquery.Client()

# # Write your SQL query
# query = """
# with my_location as (
#   SELECT  ST_GEOGPOINT(-83.94, 35.97) as my_location,
#           'Knoxville' as home
# ), 

# stations as (
#   SELECT *, ST_GEOGPOINT(lon,lat) as latlon_geo
#   FROM `bigquery-public-data.noaa_gsod.stations` 
# ), 

# get_closest as (
#   SELECT home,my_location, st.*, 
#   FROM (
#     SELECT 
#            ST_ASTEXT(my_location) as my_location, 
#            home,
#            ARRAY_AGG( # get the closest station
#               STRUCT(usaf,wban,name,lon,lat,country,state,
#                     ST_DISTANCE(my_location, b.latlon_geo)*0.00062137 as miles)
#            ) as stations
#     FROM my_location a, stations b
#     WHERE ST_DWITHIN(my_location, b.latlon_geo, 32187)  --meters = 20 miles
#     GROUP BY my_location, home
#   ), UNNEST(stations) as st
#   WHERE st.miles = (  -- Filter for station with minimum distance
#     SELECT MIN(miles) FROM UNNEST(stations)
#   )
# )
# SELECT  gc.*, 
#     gs.temp AS temperature,
#     gs.wdsp AS wind_speed,
#     gs.gust AS wind_gust,
#     gs.mxpsd AS max_wind_speed,
#     gs.prcp AS precipitation
# FROM get_closest gc, `bigquery-public-data.noaa_gsod.gsod20*` gs
# WHERE max != 9999.9  -- code for missing data
# AND   DATE(gs.date) BETWEEN '2023-01-01' AND '2023-01-31'  -- Date range for 2023
# AND   gc.usaf = gs.stn  
# AND   gc.wban = gs.wban
# ORDER BY miles ASC;
# """

# # Execute the query
# query_job = client.query(query)

# # Fetch results into a DataFrame
# result_df = query_job.to_dataframe()

# # Display the DataFrame
# print(result_df)


In [17]:
# build a function to fetch weather data 

def fetch_weather_data(latitude, longitude, date_range):
    # Define the SQL query to fetch weather data for the given location and date range
    
    query = f"""
    with sensor_location as (
    SELECT  ST_GEOGPOINT({longitude}, {latitude}) as sensor_location
    ), 

    stations as (
    SELECT *, ST_GEOGPOINT(lon,lat) as latlon_geo
    FROM `bigquery-public-data.noaa_gsod.stations` 
    ), 

    get_closest as (
    SELECT sensor_location, st.*, 
    FROM (
        SELECT 
            ST_ASTEXT(sensor_location) as sensor_location,
            ARRAY_AGG( # get the closest station
                STRUCT(usaf,wban,name,lon,lat,country,state,
                        ST_DISTANCE(sensor_location, b.latlon_geo)*0.00062137 as miles)
            ) as stations
        FROM sensor_location a, stations b
        WHERE ST_DWITHIN(sensor_location, b.latlon_geo, 32187)  --meters = 20 miles
        GROUP BY sensor_location
    ), UNNEST(stations) as st
    WHERE st.miles = (  -- Filter for station with minimum distance
        SELECT MIN(miles) FROM UNNEST(stations)
    )
    )
    SELECT  gc.*, 
        gs.temp AS temperature,
        gs.wdsp AS wind_speed,
        gs.gust AS wind_gust,
        gs.mxpsd AS max_wind_speed,
        gs.prcp AS precipitation
    FROM get_closest gc, `bigquery-public-data.noaa_gsod.gsod20*` gs
    WHERE max != 9999.9  -- code for missing data
    AND   DATE(gs.date) BETWEEN '{date_range[0]}' AND '{date_range[1]}'
    AND   gc.usaf = gs.stn  
    AND   gc.wban = gs.wban
    ORDER BY miles ASC;
    """

    # Execute the query
    query_job = client.query(query)

    # Fetch results into a DataFrame
    result_df = query_job.to_dataframe()

    return result_df

In [15]:
date_range = ('2023-01-01', '2023-01-03')

In [18]:
for index, row in sensor_data.iterrows():
    
    weather_data = fetch_weather_data(row['latitude'], row['longitude'], date_range)
    print(weather_data)

            sensor_location    usaf   wban                           name  \
0  POINT(-96.5993 32.99084)  721044  00448  RALPH M HALL ROCKWALL MUNICIP   
1  POINT(-96.5993 32.99084)  721044  00448  RALPH M HALL ROCKWALL MUNICIP   
2  POINT(-96.5993 32.99084)  721044  00448  RALPH M HALL ROCKWALL MUNICIP   

      lon     lat country state      miles  temperature wind_speed  wind_gust  \
0 -96.435  32.931      US    TX  10.383485         64.6        7.0       17.1   
1 -96.435  32.931      US    TX  10.383485         68.1        8.7       27.0   
2 -96.435  32.931      US    TX  10.383485         62.4       10.9       19.0   

  max_wind_speed  precipitation  
0           12.0            0.0  
1           19.0            0.0  
2           15.9            0.0  
Empty DataFrame
Columns: [sensor_location, usaf, wban, name, lon, lat, country, state, miles, temperature, wind_speed, wind_gust, max_wind_speed, precipitation]
Index: []
              sensor_location    usaf   wban             na

In [5]:
sensor_data

,sensor_index,name,latitude,longitude
0,2644,Sachse Farms,32.990840,-96.599300
1,9504,Rosebud,32.419700,-97.008156
2,12969,Meadow Glen,32.991665,-96.859150
3,13013,GPTX,32.766370,-97.037926
4,16271,Arbormont Estates,32.882004,-97.084130
...,...,...,...,...
65,182041,CleanAIRE NC Hickory Creek,33.126100,-97.059390
66,184053,Creekwood Estates,32.909042,-97.124160
67,196323,Duck Creek Air Quality,32.960114,-96.692200
68,196421,Kilmichael Lane - PAII,32.977444,-96.776110


In [2]:
# Import necessary library
from google.cloud import bigquery

# Create a client object
client = bigquery.Client()

def fetch_weather_station(latitude, longitude, date_range):
    # Define the SQL query to fetch weather data for the given location and date range
    query = f"""
    WITH sensor_location AS (
        SELECT ST_GEOGPOINT({longitude}, {latitude}) AS sensor_location
    ), 
    stations AS (
        SELECT *, ST_GEOGPOINT(lon,lat) AS latlon_geo
        FROM `bigquery-public-data.noaa_gsod.stations`
    )
    SELECT sensor_location, st.*
    FROM (
        SELECT 
            ST_ASTEXT(sensor_location) AS sensor_location,
            ARRAY_AGG( -- get the closest station
                STRUCT(usaf, wban, name, lon, lat, country, state,
                        ST_DISTANCE(sensor_location, b.latlon_geo) * 0.00062137 AS miles)
            ) AS stations
        FROM sensor_location a, stations b
        WHERE ST_DWITHIN(sensor_location, b.latlon_geo, 48280)  -- meters = 20 miles
        GROUP BY sensor_location
    ), UNNEST(stations) AS st
    WHERE st.miles = ( -- Filter for station with minimum distance
        SELECT MIN(miles) FROM UNNEST(stations)
    )
    """

    # Execute the query
    query_job = client.query(query)

    # Fetch results into a DataFrame
    result_df = query_job.to_dataframe()

    return result_df


In [3]:
date_range = ('2023-01-01', '2023-01-03')

In [5]:
import sqlite3
import pandas as pd
query = """
    SELECT * FROM sensor_table
    """
with sqlite3.connect('../datasets/dallas.sqlite') as db:
    sensor_data = pd.read_sql(query, db)

In [6]:
sensor_data

,sensor_index,name,latitude,longitude
0,2644,Sachse Farms,32.990840,-96.599300
1,9504,Rosebud,32.419700,-97.008156
2,12969,Meadow Glen,32.991665,-96.859150
3,13013,GPTX,32.766370,-97.037926
4,16271,Arbormont Estates,32.882004,-97.084130
...,...,...,...,...
63,182041,CleanAIRE NC Hickory Creek,33.126100,-97.059390
64,184053,Creekwood Estates,32.909042,-97.124160
65,196323,Duck Creek Air Quality,32.960114,-96.692200
66,196421,Kilmichael Lane - PAII,32.977444,-96.776110


In [12]:
# for index, row in sensor_data.iterrows():
    
#     weather_stations = fetch_weather_station(row['latitude'], row['longitude'], date_range)
#     print(weather_stations)

In [7]:
date_range = ('2023-01-01', '2023-01-05')
weather_stations_combined = pd.DataFrame()  # Initialize an empty DataFrame to store combined results

for index, row in sensor_data.iterrows():
    weather_stations = fetch_weather_station(row['latitude'], row['longitude'], date_range)
    weather_stations_combined = pd.concat([weather_stations_combined, weather_stations])

weather_stations_combined


,sensor_location,usaf,wban,name,lon,lat,country,state,miles
0,POINT(-96.5993 32.99084),721044,00448,RALPH M HALL ROCKWALL MUNICIP,-96.435,32.931,US,TX,10.383485
0,POINT(-97.008156 32.4197),720299,99999,MIDLOTHIAN/WAXAHATC,-96.917,32.450,US,TX,5.713148
0,POINT(-96.85915 32.991665),722598,03970,ADDISON AIRPORT,-96.836,32.969,US,TX,2.062202
0,POINT(-97.037926 32.76637),999999,13907,DALLAS HENSLEY FIELD AFB,-96.967,32.750,US,TX,4.273535
0,POINT(-97.08413 32.882004),999999,03927,DALLAS-FORT WORTH INTL AP,-97.019,32.898,US,TX,3.937066
...,...,...,...,...,...,...,...,...,...
0,POINT(-97.12416 32.909042),999999,03927,DALLAS-FORT WORTH INTL AP,-97.019,32.898,US,TX,6.147833
1,POINT(-97.12416 32.909042),722590,03927,DALLAS/FT WORTH INTERNATIONAL,-97.019,32.898,US,TX,6.147833
0,POINT(-96.6922 32.960114),722598,99999,ADDISON,-96.833,32.967,US,TX,8.176092
0,POINT(-96.77611 32.977444),722598,99999,ADDISON,-96.833,32.967,US,TX,3.375646


In [8]:
weather_stations_combined.to_csv('../datasets/weather_stations.csv')

In [95]:
import pandas as pd
import sqlite3
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

weather_stations_combined = pd.DataFrame()
# Function to fetch weather stations
def fetch_weather_station(latitude, longitude):
    # Define the SQL query to fetch weather data for the given location and date range
    query = f"""
    WITH sensor_location AS (
        SELECT ST_GEOGPOINT({longitude}, {latitude}) AS sensor_location
    ),
    stations AS (
        SELECT *,
            ST_GEOGPOINT(lon, lat) AS latlon_geo,
            ST_DISTANCE(ST_GEOGPOINT({longitude}, {latitude}), ST_GEOGPOINT(lon, lat)) AS distance
        FROM `bigquery-public-data.noaa_gsod.stations`
    ),
    ranked_stations AS (
        SELECT *,
            ROW_NUMBER() OVER (ORDER BY distance) AS rank
        FROM stations
    )
    SELECT 
        ST_ASTEXT(sl.sensor_location) AS sensor_location, 
        rs.*
    FROM 
        sensor_location sl
    JOIN 
        ranked_stations rs ON rs.rank <= 3
    ORDER BY 
        rs.distance


    """

    # Execute the query
    query_job = client.query(query)
    # Fetch results into a DataFrame
    result_df = query_job.to_dataframe()
    return result_df

# Connect to SQLite database and fetch sensor data
query = """
    SELECT * FROM sensor_table
    """
with sqlite3.connect('../datasets/dallas.sqlite') as db:
    sensor_data = pd.read_sql(query, db)

    # Iterate over sensor locations and fetch closest weather stations
    for index, row in sensor_data.iterrows():
        latitude = row['latitude']
        longitude = row['longitude']
        weather_stations = fetch_weather_station(latitude, longitude)
        print(weather_stations)
        # Concatenate weather station data
        weather_stations_combined = pd.concat([weather_stations_combined, weather_stations])

            sensor_location    usaf   wban                           name  \
0  POINT(-96.5993 32.99084)  008411  99999  XM20                            
1  POINT(-96.5993 32.99084)  008414  99999  XM18                            
2  POINT(-96.5993 32.99084)  008415  99999  XM21                            

  country state  call  lat  lon  elev     begin       end latlon_geo  \
0    None  None  None  NaN  NaN  None  20160217  20160217       None   
1    None  None  None  NaN  NaN  None  20160216  20160217       None   
2    None  None  None  NaN  NaN  None  20160217  20200314       None   

   distance  rank  
0       NaN     1  
1       NaN     2  
2       NaN     3  
             sensor_location    usaf   wban                           name  \
0  POINT(-97.008156 32.4197)  008411  99999  XM20                            
1  POINT(-97.008156 32.4197)  008414  99999  XM18                            
2  POINT(-97.008156 32.4197)  008415  99999  XM21                            

  country

In [69]:
sensor_data

,sensor_index,name,latitude,longitude
0,2644,Sachse Farms,32.990840,-96.599300
1,9504,Rosebud,32.419700,-97.008156
2,12969,Meadow Glen,32.991665,-96.859150
3,13013,GPTX,32.766370,-97.037926
4,16271,Arbormont Estates,32.882004,-97.084130
...,...,...,...,...
63,182041,CleanAIRE NC Hickory Creek,33.126100,-97.059390
64,184053,Creekwood Estates,32.909042,-97.124160
65,196323,Duck Creek Air Quality,32.960114,-96.692200
66,196421,Kilmichael Lane - PAII,32.977444,-96.776110


In [96]:
weather_stations_combined

,sensor_location,usaf,wban,name,country,state,call,lat,lon,elev,begin,end,latlon_geo,distance,rank
0,POINT(-96.5993 32.99084),008411,99999,XM20,None,None,None,NaN,NaN,None,20160217,20160217,None,NaN,1
1,POINT(-96.5993 32.99084),008414,99999,XM18,None,None,None,NaN,NaN,None,20160216,20160217,None,NaN,2
2,POINT(-96.5993 32.99084),008415,99999,XM21,None,None,None,NaN,NaN,None,20160217,20200314,None,NaN,3
0,POINT(-97.008156 32.4197),008411,99999,XM20,None,None,None,NaN,NaN,None,20160217,20160217,None,NaN,1
1,POINT(-97.008156 32.4197),008414,99999,XM18,None,None,None,NaN,NaN,None,20160216,20160217,None,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,POINT(-96.77611 32.977444),008414,99999,XM18,None,None,None,NaN,NaN,None,20160216,20160217,None,NaN,2
2,POINT(-96.77611 32.977444),008415,99999,XM21,None,None,None,NaN,NaN,None,20160217,20200314,None,NaN,3
0,POINT(-96.98121 32.53494),008411,99999,XM20,None,None,None,NaN,NaN,None,20160217,20160217,None,NaN,1
1,POINT(-96.98121 32.53494),008414,99999,XM18,None,None,None,NaN,NaN,None,20160216,20160217,None,NaN,2


In [35]:
weather_stations_combined_dropped = weather_stations_combined.drop_duplicates(subset=['sensor_location', 'distance'])

In [70]:
weather_stations_combined

,sensor_location,usaf,wban,name,country,state,call,lat,lon,elev,begin,end,latlon_geo,distance
0,POINT(-96.5993 32.99084),721044,00448,RALPH M HALL ROCKWALL MUNICIP,US,TX,KF46,32.931,-96.435,+0175.3,20120813,20210921,POINT(-96.435 32.931),16710.630615
0,POINT(-97.008156 32.4197),720299,99999,MIDLOTHIAN/WAXAHATC,US,TX,KJWY,32.450,-96.917,+0217.0,20040525,20051231,POINT(-96.917 32.45),9194.437587
0,POINT(-96.85915 32.991665),722598,03970,ADDISON AIRPORT,US,TX,KADS,32.969,-96.836,+0196.3,20060101,20210921,POINT(-96.836 32.969),3318.798729
0,POINT(-97.037926 32.76637),999999,13907,DALLAS HENSLEY FIELD AFB,US,TX,None,32.750,-96.967,+0149.0,19490101,19520421,POINT(-96.967 32.75),6877.601853
0,POINT(-97.08413 32.882004),999999,03927,DALLAS-FORT WORTH INTL AP,US,TX,KDFW,32.898,-97.019,+0181.7,19530501,19721231,POINT(-97.019 32.898),6336.106405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,POINT(-97.12416 32.909042),999999,03927,DALLAS-FORT WORTH INTL AP,US,TX,KDFW,32.898,-97.019,+0181.7,19530501,19721231,POINT(-97.019 32.898),9893.996553
1,POINT(-97.12416 32.909042),722590,03927,DALLAS/FT WORTH INTERNATIONAL,US,TX,KDFW,32.898,-97.019,+0170.7,19730101,20210921,POINT(-97.019 32.898),9893.996553
0,POINT(-96.6922 32.960114),722598,99999,ADDISON,US,TX,KADS,32.967,-96.833,+0196.0,19740201,20051231,POINT(-96.833 32.967),13158.170097
0,POINT(-96.77611 32.977444),722598,99999,ADDISON,US,TX,KADS,32.967,-96.833,+0196.0,19740201,20051231,POINT(-96.833 32.967),5432.585886


In [71]:
weather_stations_combined = weather_stations_combined[['name', 'usaf', 'wban', 'lat', 'lon', 'distance']].rename(columns = {
    'name': 'stations_name',
    'lat': 'stations_lat',
    'lon': 'stations_lon',
    'distance': 'stations_distance'
})

In [84]:
weather_stations_combined

,stations_name,usaf,wban,stations_lat,stations_lon,stations_distance
0,RALPH M HALL ROCKWALL MUNICIP,721044,00448,32.931,-96.435,16710.630615
0,MIDLOTHIAN/WAXAHATC,720299,99999,32.450,-96.917,9194.437587
0,ADDISON AIRPORT,722598,03970,32.969,-96.836,3318.798729
0,DALLAS HENSLEY FIELD AFB,999999,13907,32.750,-96.967,6877.601853
0,DALLAS-FORT WORTH INTL AP,999999,03927,32.898,-97.019,6336.106405
...,...,...,...,...,...,...
0,DALLAS-FORT WORTH INTL AP,999999,03927,32.898,-97.019,9893.996553
1,DALLAS/FT WORTH INTERNATIONAL,722590,03927,32.898,-97.019,9893.996553
0,ADDISON,722598,99999,32.967,-96.833,13158.170097
0,ADDISON,722598,99999,32.967,-96.833,5432.585886


In [73]:
sensor_data

,sensor_index,name,latitude,longitude
0,2644,Sachse Farms,32.990840,-96.599300
1,9504,Rosebud,32.419700,-97.008156
2,12969,Meadow Glen,32.991665,-96.859150
3,13013,GPTX,32.766370,-97.037926
4,16271,Arbormont Estates,32.882004,-97.084130
...,...,...,...,...
63,182041,CleanAIRE NC Hickory Creek,33.126100,-97.059390
64,184053,Creekwood Estates,32.909042,-97.124160
65,196323,Duck Creek Air Quality,32.960114,-96.692200
66,196421,Kilmichael Lane - PAII,32.977444,-96.776110


In [91]:
# Function to fetch weather data for a given station
def fetch_weather_data(usaf, wban, start_date, end_date):
    # Define the SQL query to fetch weather data for the given station and date range
    query = f"""
    SELECT 
        stn AS usaf,
        wban,
        COUNT(temp) AS data_points,
    FROM `bigquery-public-data.noaa_gsod.gsod20*`
    WHERE stn = '{usaf}'
    AND wban = '{wban}'
    AND DATE(date) BETWEEN '{start_date}' AND '{end_date}'
    AND max != 9999.9  -- code for missing data
    GROUP BY stn, wban
    ORDER BY data_points ASC
    """

    # Execute the query
    query_job = client.query(query)
    # Fetch results into a DataFrame
    result_df = query_job.to_dataframe()
    return result_df

In [92]:
start_date = '2023-01-01'
end_date = '2023-01-03'

weather_data_combined = pd.DataFrame()

# Iterate over sensor locations and fetch weather data for the nearest stations
for index, row in weather_stations_combined.iterrows():
    usaf = row['usaf']
    wban = row['wban']
    weather_data = fetch_weather_data(usaf, wban, start_date, end_date)
    print(weather_data)
    # weather_data['sensor_index'] = row['sensor_index']  # Add sensor index to weather data
    weather_data_combined = pd.concat([weather_data_combined, weather_data], ignore_index=True)

     usaf   wban  data_points
0  721044  00448            3
Empty DataFrame
Columns: [usaf, wban, data_points]
Index: []


/tmp/ipykernel_69461/1677853529.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_data_combined = pd.concat([weather_data_combined, weather_data], ignore_index=True)


     usaf   wban  data_points
0  722598  03970            3
Empty DataFrame
Columns: [usaf, wban, data_points]
Index: []


/tmp/ipykernel_69461/1677853529.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_data_combined = pd.concat([weather_data_combined, weather_data], ignore_index=True)


Empty DataFrame
Columns: [usaf, wban, data_points]
Index: []


...

     usaf   wban  data_points
0  722590  03927            3
Empty DataFrame
Columns: [usaf, wban, data_points]
Index: []


/tmp/ipykernel_69461/1677853529.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_data_combined = pd.concat([weather_data_combined, weather_data], ignore_index=True)


Empty DataFrame
Columns: [usaf, wban, data_points]
Index: []


/tmp/ipykernel_69461/1677853529.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_data_combined = pd.concat([weather_data_combined, weather_data], ignore_index=True)


     usaf   wban  data_points
0  720299  53966            3


In [94]:
weather_data_combined

,usaf,wban,data_points
0,721044,00448,3
1,722598,03970,3
2,722590,03927,3
3,722594,53909,3
4,747390,13961,3
5,722598,03970,3
6,722598,03970,3
7,722589,03991,3
8,722589,03991,3
9,722598,03970,3


In [50]:
sensor_data.reset_index(drop=True, inplace=True)
weather_stations_combined.reset_index(drop=True, inplace=True)

merged_data = pd.concat([sensor_data, weather_stations_combined], axis=1)
# Commit the changes to the database
# merged_data.to_sql('sensor_data', db, if_exists='replace', index=False)

In [51]:
merged_data.head()

,sensor_index,name,latitude,longitude,stations_name,usaf,wban,stations_lat,stations_lon,stations_distance
0,2644.0,Sachse Farms,32.990840,-96.599300,RALPH M HALL ROCKWALL MUNICIP,721044,00448,32.931,-96.435,16710.630615
1,9504.0,Rosebud,32.419700,-97.008156,MIDLOTHIAN/WAXAHATC,720299,99999,32.450,-96.917,9194.437587
2,12969.0,Meadow Glen,32.991665,-96.859150,ADDISON AIRPORT,722598,03970,32.969,-96.836,3318.798729
3,13013.0,GPTX,32.766370,-97.037926,DALLAS HENSLEY FIELD AFB,999999,13907,32.750,-96.967,6877.601853
4,16271.0,Arbormont Estates,32.882004,-97.084130,DALLAS-FORT WORTH INTL AP,999999,03927,32.898,-97.019,6336.106405


In [52]:
# with sqlite3.connect('../datasets/dallas.sqlite') as db:
#     merged_data.to_sql('sensor_table', db, if_exists='replace', index=False)

In [81]:
query = """
    SELECT * FROM sensor_table
    """
with sqlite3.connect('../datasets/dallas.sqlite') as db:
    sensor_data = pd.read_sql(query, db)

In [82]:
sensor_data

,sensor_index,name,latitude,longitude
0,2644,Sachse Farms,32.990840,-96.599300
1,9504,Rosebud,32.419700,-97.008156
2,12969,Meadow Glen,32.991665,-96.859150
3,13013,GPTX,32.766370,-97.037926
4,16271,Arbormont Estates,32.882004,-97.084130
...,...,...,...,...
63,182041,CleanAIRE NC Hickory Creek,33.126100,-97.059390
64,184053,Creekwood Estates,32.909042,-97.124160
65,196323,Duck Creek Air Quality,32.960114,-96.692200
66,196421,Kilmichael Lane - PAII,32.977444,-96.776110


In [64]:
# #IF i ever need to drop a table
# import sqlite3

# # Connect to the SQLite database
# conn = sqlite3.connect('../datasets/dallas.sqlite')
# cursor = conn.cursor()

# # Execute the SQL command to drop the table
# cursor.execute("DROP TABLE IF EXISTS sensor_table")

# # Commit the changes
# conn.commit()

# # Close the connection
# conn.close()


In [80]:
import pandas as pd
import sqlite3
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

# Connect to SQLite database and fetch sensor data
query = "SELECT * FROM sensor_table"
with sqlite3.connect('../datasets/dallas.sqlite') as db:
    sensor_data = pd.read_sql(query, db)

# Function to fetch weather data for a given station
def fetch_weather_data(usaf, wban, start_date, end_date):
    # Define the SQL query to fetch weather data for the given station and date range
    query = f"""
    SELECT 
        stn AS usaf,
        wban,
        date,
        temp AS temperature,
        wdsp AS wind_speed,
        gust AS wind_gust,
        mxpsd AS max_wind_speed,
        prcp AS precipitation
    FROM `bigquery-public-data.noaa_gsod.gsod20*`
    WHERE stn = '{usaf}'
    AND wban = '{wban}'
    AND DATE(date) BETWEEN '{start_date}' AND '{end_date}'
    AND max != 9999.9  -- code for missing data
    ORDER BY date ASC
    """

    # Execute the query
    query_job = client.query(query)
    # Fetch results into a DataFrame
    result_df = query_job.to_dataframe()
    return result_df

# Define the date range for fetching weather data
start_date = '2023-01-01'
end_date = '2023-01-03'

weather_data_combined = pd.DataFrame()

# Iterate over sensor locations and fetch weather data for the nearest stations
for index, row in sensor_data.iterrows():
    usaf = row['usaf']
    wban = row['wban']
    weather_data = fetch_weather_data(usaf, wban, start_date, end_date)
    print(weather_data)
    # weather_data['sensor_index'] = row['sensor_index']  # Add sensor index to weather data
    # weather_data_combined = pd.concat([weather_data_combined, weather_data], ignore_index=True)

KeyError: 'usaf'

In [61]:
weather_data_combined

,usaf,wban,date,temperature,wind_speed,wind_gust,max_wind_speed,precipitation,sensor_index
0,721044,00448,2023-01-01,62.4,10.9,19.0,15.9,0.0,2644.0
1,721044,00448,2023-01-02,68.1,8.7,27.0,19.0,0.0,2644.0
2,721044,00448,2023-01-03,64.6,7.0,17.1,12.0,0.0,2644.0
3,722598,03970,2023-01-01,65.2,7.6,19.0,12.0,0.0,12969.0
4,722598,03970,2023-01-02,69.0,9.6,22.9,17.1,0.0,12969.0
...,...,...,...,...,...,...,...,...,...
85,722598,03970,2023-01-02,69.0,9.6,22.9,17.1,0.0,182041.0
86,722598,03970,2023-01-03,63.8,6.0,18.1,12.0,0.0,182041.0
87,720299,53966,2023-01-01,61.4,6.4,17.1,12.0,0.0,NaN
88,720299,53966,2023-01-02,68.1,8.7,27.0,20.0,0.0,NaN
